In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import time

In [2]:
in_csv = 'files/weather_train.csv'
out_csv = 'files/weather_train_fixweather.csv'
yrrange = 1
yrref = 2016

# in_csv = 'files/weather_test.csv'
# out_csv = 'files/weather_test_fixweather.csv'
# yrrange = 2
# yrref = 2017

weather = pd.read_csv(in_csv)
weather2 = weather.copy()

In [3]:
display(weather.head())

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [4]:
# weather2['cloud_coverage'] = weather2['cloud_coverage'].fillna(value=0.0)

In [5]:
missing_val_count_by_column = (weather2.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64


In [6]:
weather.shape

(139773, 9)

## timestamp range

In [7]:
timestamp = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in weather['timestamp'] ]
year = [x.year for x in timestamp]
month = [x.month - 1 for x in timestamp]
hour = [x.hour for x in timestamp]
weekday = [x.weekday() for x in timestamp]
# hour = [int(x/3) for x in hour]
weather2.insert(0, 'year', year)
weather2.insert(0, 'month', month)
weather2.insert(0, 'hour', hour)
weather2.insert(0, 'weekday', weekday)

In [8]:
# weather2.month = weather2.month -1

In [9]:
display(weather2.head())

,weekday,hour,month,year,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,4,0,0,2016,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,4,1,0,2016,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,4,2,0,2016,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,4,3,0,2016,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,4,4,0,2016,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [10]:
# get average , fix site_id season and hour

In [11]:
avg = np.array((yrrange,16,12,24))
# change to 1
avg = [[[ [0 for col in range(24)] for col in range(12)] for row in range(16)] for row in range(yrrange)]
for yr in range(0,yrrange):
    for site_id in range(0,16):
        for month in range(0,12):
            for hour in range(0,24):
                t = weather2[(weather2['site_id'] == site_id) & (weather2['year'] == yrref + yr) & (weather2['month'] == month) & (weather2['hour'] == hour)]
#                 print(t)
                avg[yr][site_id][month][hour] = t.mean()
    #             break
    #         break
    #     break


### fix air_temperature, dew_temperature, wind_speed

In [12]:
label_list = ['air_temperature','dew_temperature','wind_speed']

In [13]:
cnt = 0
for label in label_list:
    a = weather2[weather2[label].isna()]
    for index, row in a.iterrows():
        cnt = cnt +1
        (site_id, year, month, hour) = row[['site_id', 'year', 'month', 'hour']]
        yearno = year - yrref
#         print(site_id, season, hour)
        for interval in (1,3,12,24):
            t = weather2.iloc[index - interval:index + interval + 1]
            t2 = t[label]
            t = t[~t2.isna()]
            if (t2.isna().sum()<len(t2)/2) | (interval == 24) :
                ref = list()
                for i, r in t.iterrows():
                    ref.append(r[label] - avg[yearno][site_id][month][r['hour']][label])
#                 print(ref)
                ref = sum(ref) / len(ref) 
                weather2.at[index,label] = round(ref + avg[yearno][site_id][month][hour][label],1)
#                 print(weather2.at[index,label])
                done = 1
                break
#                 relax constraint
#         if done == 0:
#             if t2.isna().sum()<len(t2):
#                 weather2.at[index,label] = round(t2.mean(),1)
#                 continue
        if done==0:
            print('Error')
weather2.loc[weather2['wind_speed'] < 0,'wind_speed'] = 0
print(cnt)

472


### fix sea_level_pressure, ignore some site_id

In [14]:
label_list = ['sea_level_pressure']
ignore_sea = [5]

In [15]:
cnt = 0
for label in label_list:
    a = weather2[weather2[label].isna()]
    a = a[~a['site_id'].isin(ignore_sea)]
#     weather2.loc[weather2['site_id'].isin(ignore_sea),label] = 0
    for index, row in a.iterrows():
        cnt = cnt +1
        (site_id, year, month, hour) = row[['site_id', 'year', 'month', 'hour']]
        yearno = year - yrref
#         print(site_id, season, hour)
        for interval in (1,3,12,24):
            t = weather2.iloc[index - interval:index + interval + 1]
            t2 = t[label]
            t = t[~t2.isna()]
            if (t2.isna().sum()<len(t2)/2) | (interval == 24) :
                ref = list()
                for i, r in t.iterrows():
                    ref.append(r[label] - avg[yearno][site_id][month][r['hour']][label])
#                 print(ref)
                ref = sum(ref) / len(ref) 
                weather2.at[index,label] = round(ref + avg[yearno][site_id][month][hour][label],1)
#                 print(weather2.at[index,label])
                done = 1
                break
#                 relax constraint
#         if done == 0:
#             if t2.isna().sum()<len(t2):
#                 weather2.at[index,label] = round(t2.mean(),1)
#                 continue
        if done==0:
            print('Error')
print(cnt)

1863


In [16]:
t = weather2.groupby('timestamp')['sea_level_pressure'].mean()

In [17]:
t = weather2[weather2['site_id'].isin(ignore_sea)]['timestamp'].apply(lambda x:t[x])

In [18]:
weather2.loc[weather2['site_id'].isin(ignore_sea),'sea_level_pressure'] = t

### fix precip_depth_1_hr, ignore some site_id

In [19]:
label_list = ['precip_depth_1_hr']
ignore_precip = [1,5,7,11,12,15]

In [20]:
cnt = 0
weather2.loc[weather2['precip_depth_1_hr'] == -1,'precip_depth_1_hr'] = 0.5
for label in label_list:
    a = weather2[weather2[label].isna()]
    a = a[~a['site_id'].isin(ignore_precip)]
    weather2.loc[weather2['site_id'].isin(ignore_precip),label] = 0
    for index, row in a.iterrows():
        cnt = cnt +1
        (site_id, year, month, hour) = row[['site_id', 'year', 'month', 'hour']]
        yearno = year - yrref
#         print(site_id, season, hour)
        for interval in (1,3,12,24):
            t = weather2.iloc[index - interval:index + interval + 1]
            t2 = t[label]
            t = t[~t2.isna()]
            if (t2.isna().sum()<len(t2)/2) | (interval == 24) :
                weather2.at[index,label] = t2.mean()
                done = 1
                break
        if done==0:
            print('Error')
weather2.loc[weather2['precip_depth_1_hr'] <0.2, 'precip_depth_1_hr'] = 0
weather2['precip_depth_1_hr'].fillna(value =0, inplace = True)
print(cnt)

1499


### fix wind dir

In [21]:
def fill_wind_dir(t):
    if t.size == 1:
        return min(t)
    t1 = min(t)
    t2 = max(t)
    if(t2-t1 >= 180):
        return int((t1+t2+360)/2) % 360
    return int((t1+t2)/2)

a = weather[weather['wind_direction'].isna()]

for index, row in a.iterrows():
    t = weather.iloc[index-1:index+2]
    t = t['wind_direction']
#     print(t)
    if t.isna().sum()<len(t):
        weather2.loc[index,'wind_direction'] = fill_wind_dir(t[~t.isna()])
# if no nearby data        
weather2['wind_direction'] = weather2['wind_direction'].fillna(method='ffill')

### add relative_humidity,apparent_temp

In [22]:
def relative_humidity(Tc,Tdc):
    E = 6.11*10.0**(7.5*Tdc/(237.7+Tdc))
    Es = 6.11*10.0**(7.5*Tc/(237.7+Tc))    
    RH = (E/Es)*100
    return RH

weather2['humidity'] = relative_humidity(weather2.air_temperature, weather2.dew_temperature).astype(np.float16)
weather2.loc[weather2['humidity']>100, 'humidity'] = 100

In [23]:
def apparent_temp(T,v):
    return T-2*np.sqrt(v)
#     return RH
# t = apparent_temp(weather2.air_temperature, weather2.wind_speed).astype(np.float16)
weather2['apparent_temp'] = apparent_temp(weather2.air_temperature, weather2.wind_speed).astype(np.float16)

In [24]:
missing_val_count_by_column = (weather2.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

cloud_coverage    69173
dtype: int64


In [25]:
# weather2.drop('year', axis=1 ,inplace = True)
display(weather2)

,weekday,hour,month,year,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,humidity,apparent_temp
0,4,0,0,2016,0,2016-01-01 00:00:00,25.0,6.0,20.0,0.0,1019.7,0.0,0.0,73.87500,25.000000
1,4,1,0,2016,0,2016-01-01 01:00:00,24.4,NaN,21.1,0.5,1020.2,70.0,1.5,81.87500,21.953125
2,4,2,0,2016,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,90.18750,22.796875
3,4,3,0,2016,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,97.00000,21.093750
4,4,4,0,2016,0,2016-01-01 04:00:00,20.0,2.0,20.0,0.5,1020.0,250.0,2.6,100.00000,16.781250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139768,5,19,11,2016,15,2016-12-31 19:00:00,3.0,NaN,-8.0,0.0,1007.8,180.0,5.7,44.18750,-1.775391
139769,5,20,11,2016,15,2016-12-31 20:00:00,2.8,2.0,-8.9,0.0,1007.4,180.0,7.7,41.78125,-2.750000
139770,5,21,11,2016,15,2016-12-31 21:00:00,2.8,NaN,-7.2,0.0,1007.5,180.0,5.1,47.68750,-1.716797
139771,5,22,11,2016,15,2016-12-31 22:00:00,2.2,NaN,-6.7,0.0,1008.0,170.0,4.6,51.71875,-2.089844


In [26]:
weather2.to_csv(out_csv, index=False)